In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras import preprocessing , utils
import os
import yaml
import json
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

In [ ]:
docs=open(r'DATASET.txt',encoding='utf-8').read().split("\n")
c=docs[0].strip().split("\t")

In [ ]:
questions_for_token = list()
answers_for_token = list()
c=1
for con in docs:
    if(c==2868):
        pass
    else:
        con=con.strip().split("\t")
        questions_for_token.append(con[0])
        answers_for_token.append(con[1])
        c+=1

In [4]:
embed_size = 100 # how big is each word vector
max_features=6000
maxlen=100

In [5]:
import re
def processTweet(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\.!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    return pattern.sub(r"\1", s)
def getFeatureVector(chat):
    chat=processTweet(chat)
    featureVector = []
    #split tweet into words
    words = chat.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(val is None):
            continue
        else:
            featureVector.append(w.lower())
    return " ".join(list(featureVector))

In [6]:
def emb_mat(nb_words):
    EMBEDDING_FILE="glove.6B.100d.txt"
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    emb_mean,emb_std

    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words+1, embed_size))
    for word, i in word_index.items():
        if (i >= max_features) or i==nb_words:
            continue
        embedding_vector = embeddings_index.get(word) #here we will get embedding for each word from GloVe
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [7]:
def tokenized_data(questions,answers,VOCAB_SIZE,tokenizer):
    # encoder_input_data
    import numpy as np
    tokenized_questions = tokenizer.texts_to_sequences( questions )
    maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
    padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen , padding='post' )
    encoder_input_data = np.array( padded_questions )
    #print( encoder_input_data.shape , maxlen_questions )

    # decoder_input_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    decoder_input_data = np.array( padded_answers )
    #print( decoder_input_data.shape , maxlen_answers )

    # decoder_output_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    for i in range(len(tokenized_answers)) :
        tokenized_answers[i] = tokenized_answers[i][1:] # remove <start> take rest
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE)
    decoder_output_data = np.array( onehot_answers )
    #print( decoder_output_data.shape )
    
    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_answers]

In [8]:
def enable_gpu():
    #config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
    #sess = tf.compat.v1.Session(config=config) 
    #tf.compat.v1.keras.backend.set_session(
    #    sess
    #)
    #gpu_options = tf.compat.v1.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.333)
    #session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
def get_model(nb_words,embed_size,embedding_matrix):
    encoder_inputs = tf.keras.layers.Input(shape=( None , ))
    encoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True ,weights=[embedding_matrix]) (encoder_inputs)
    encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
    encoder_states = [ state_h , state_c ]

    decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
    decoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True,weights=[embedding_matrix]) (decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

    decoder_dense = tf.keras.layers.Dense( nb_words+1 , activation=tf.keras.activations.softmax ) 
    output = decoder_dense ( decoder_outputs )
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h_d, state_c_d = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h_d, state_c_d]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
    return model,encoder_model,encoder_inputs,encoder_states,decoder_lstm,decoder_embedding,decoder_dense,decoder_inputs,decoder_outputs,decoder_model

In [9]:
# define the checkpoint
filepath = "model_Translate_new1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [21]:
def prepare_data(questions,answers):
    answers=pd.DataFrame(answers, columns=["Ans"])
    questions=pd.DataFrame(questions, columns=["Question"])
    questions["TokQues"]=questions["Question"].apply(getFeatureVector)

    answers=np.array(answers["Ans"])
    questions=np.array(questions["TokQues"])

    answers_with_tags = list()
    for i in range( len( answers ) ):
        if type( answers[i] ) == str:
            answers_with_tags.append( answers[i] )
        else:
            print(questions[i])
            print(answers[i])
            print(type(answers[i]))
            questions.pop(i)

    answers = list()
    for i in range( len( answers_with_tags ) ) :
        answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
    
    
    tokenizer = preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(questions+answers)

    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))

    #embedding_matrix=emb_mat(nb_words)[0]
    #emb_vec=emb_mat(nb_words)[1]

    VOCAB_SIZE = len( tokenizer.word_index )+1
    
    
    tok_out=tokenized_data(questions,answers,VOCAB_SIZE,tokenizer)
    encoder_input_data=tok_out[0]
    decoder_input_data=tok_out[1]
    decoder_output_data=tok_out[2]
    maxlen_answers=tok_out[3]
    
    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_answers,nb_words,word_index,tokenizer]

In [11]:
enable_gpu()

In [14]:
Prepared_data=prepare_data(questions_for_token,answers_for_token)
print("Preparation done")
encoder_input_data=Prepared_data[0]
print("Encoder done")
decoder_input_data=Prepared_data[1]
decoder_output_data=Prepared_data[2]
print("Decoder done")
maxlen_answers=Prepared_data[3]
nb_words=Prepared_data[4]
word_index=Prepared_data[5]
tokenizer=Prepared_data[6]
print(encoder_input_data.shape, decoder_input_data.shape, decoder_output_data.shape)

embedding_matrix=emb_mat(nb_words)
#model=get_model(nb_words,embed_size,embedding_matrix)[0]
#encoder_model=get_model(nb_words,embed_size,embedding_matrix)[1]
#encoder_inputs=get_model(nb_words,embed_size,embedding_matrix)[2]
#encoder_states=get_model(nb_words,embed_size,embedding_matrix)[3]
#decoder_lstm=get_model(nb_words,embed_size,embedding_matrix)[4]
#decoder_embedding=get_model(nb_words,embed_size,embedding_matrix)[5]
#decoder_dense=get_model(nb_words,embed_size,embedding_matrix)[6]
#decoder_inputs=get_model(nb_words,embed_size,embedding_matrix)[7]
#decoder_outputs=get_model(nb_words,embed_size,embedding_matrix)[8]
#decoder_model=get_model(nb_words,embed_size,embedding_matrix)[9]

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True, weights=[embedding_matrix]) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True,weights=[embedding_matrix]) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

decoder_dense = tf.keras.layers.Dense( nb_words+1 , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )


model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=20, callbacks=callbacks_list)

Preparation done
Encoder done
Decoder done
(2867, 100) (2867, 100) (2867, 100, 5375)
Train on 2867 samples
Epoch 1/20
2860/2867 [============================>.] - ETA: 0s - loss: 0.4698 - accuracy: 0.2378
Epoch 00001: loss improved from inf to 0.46957, saving model to model_Translate_new1.h5
2867/2867 [==============================] - 62s 22ms/sample - loss: 0.4696 - accuracy: 0.2379
Epoch 2/20
2860/2867 [============================>.] - ETA: 0s - loss: 0.4144 - accuracy: 0.2829
Epoch 00002: loss improved from 0.46957 to 0.41434, saving model to model_Translate_new1.h5
2867/2867 [==============================] - 29s 10ms/sample - loss: 0.4143 - accuracy: 0.2829
Epoch 3/20
2860/2867 [============================>.] - ETA: 0s - loss: 0.3898 - accuracy: 0.3123
Epoch 00003: loss improved from 0.41434 to 0.38971, saving model to model_Translate_new1.h5
2867/2867 [==============================] - 30s 10ms/sample - loss: 0.3897 - accuracy: 0.3124
Epoch 4/20
2860/2867 [====================

## For Inference

In [26]:
#from tensorflow.keras.models import load_model
#Loaded_model = load_model(r'model_Translate_new.h5')

In [16]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [17]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen , padding='post')

In [18]:
enc_model , dec_model = make_inference_models()

In [25]:
for _ in range(10):
    try:
        states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[0, 0] = tokenizer.word_index['start']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition :
            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
            sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
            sampled_word = None
            for word , index in tokenizer.word_index.items() :
                if sampled_word_index == index :
                    decoded_translation += ' {}'.format( word )
                    sampled_word = word
            
            if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True
                
            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            states_values = [ h , c ] 

        print( " ".join(decoded_translation.strip().split(" ")[:-1]) )
    except:
        print("Sorry, I don't understand. Please try again.")

Enter question : what are you doing
क्या तुम कैसे रहे हो
Enter question : how are you?
Sorry, I don't understand. Please try again.
Enter question : ow are you
Sorry, I don't understand. Please try again.
Enter question : how are you
आप कैसे हैं
Enter question : i am good
मैं अच्छा हूँ।
Enter question : what can you do for me
क्या तुम मुझे यकीन करते हो
Enter question : i do not
मैं नहीं कर सकता हूँ।
Enter question : ok do not do it
क्या कर रहा है क्या
Enter question : where do you live
तुम कहाँ जा रहे हो
Enter question : where are you going
तुम कहाँ जा रहे हो


## Still a better accuracy with 20 epoches. We can make it better with more data and training.